Quick experiment to see which is better at detecting truthful answers

- model outputs
- hs
- supressed activations (Hypothesis this is better)

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from loguru import logger
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from einops import rearrange, repeat
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.data import DataCollatorForLanguageModeling

import torch
from torch import Tensor
from torch.nn.functional import (
    binary_cross_entropy_with_logits as bce_with_logits,
)
from torch.nn.functional import (
    cross_entropy,
)
from pathlib import Path
from jaxtyping import Float
from torch import Tensor

import functools
import pandas as pd
import numpy as np

import itertools
from tqdm.auto import tqdm
import random
import json
from tqdm.auto import tqdm

from activation_store.collect import activation_store, default_postprocess_result

In [ ]:
import gc
def clear_mem():
    """
    Clear memory
    """
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    torch.cuda.synchronize()
    torch.cuda.reset_peak_memory_stats()
    return None
clear_mem()

## Load model

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# Qwen/Qwen3-1.7
# Qwen/Qwen3-0.6B-FP8
model_name = "Qwen/Qwen3-4B"
batch_size = 6

model_name = "Qwen/Qwen3-1.7B"
batch_size = 10
# model_name = "Qwen/Qwen3-8B"

# model_name = "unsloth/Llama-3.2-1B-Instruct"

# model_name = "Qwen/Qwen2.5-3B-Instruct"
# model_name = "Qwen/Qwen2.5-3B-Instruct-AWQ"

# model_name = "AMead10/Llama-3.2-3B-Instruct-AWQ"

# model_name = "unsloth/Phi-4-mini-instruct" # 4b
# model_name = "stelterlab/phi-4-AWQ"



In [ ]:

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if ('awq' not in model_name.lower()) else torch.float16,
    device_map="auto",
    attn_implementation="eager",  # flex_attention  flash_attention_2 sdpa eager
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

## Load data and tokenize

In [ ]:
# N = 316
max_length = 90
split = "train"
ds1 = load_dataset("Yik/truthfulQA-bool", split=split, keep_in_memory=False)

sys_msg = """Predict if a statement is true on wikipedia, return 0 for false and 1 for true.
"""


def postprocess_activation_ds_rows(row):
    messages = [
        {"role": "system", "content": sys_msg},
        {"role": "user", "content": row["question"]},
        {"role": "assistant", "content": "The answer is "},
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        return_dict=True,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        # add_generation_prompt=True,
        continue_final_message=True,
        padding_side="left",
        truncation_side="left",
    )


ds2 = ds1.map(postprocess_activation_ds_rows).with_format("torch")
new_cols = list(set(ds2.column_names) - set(ds1.column_names)) + ["label"]
ds2 = ds2.select_columns(new_cols)
ds2

In [ ]:
tokenizer.batch_decode(ds2['input_ids'])[0]

## Data loader

In [ ]:
collate_fn = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
ds = DataLoader(ds2, batch_size=batch_size, collate_fn=collate_fn)
print(ds)


## Collect activations

In [ ]:
# # choose layers to cache
# n_layers = model.config.num_hidden_layers
# a = int(0.3*n_layers)
# b = n_layers-2
# layer_groups = {
#     'mlp.down_proj': [k for k,v in model.named_modules() if k.endswith('mlp.down_proj')][a:b],
#     'self_attn': [k for k,v in model.named_modules() if k.endswith('.self_attn')][a:b],
#     'mlp.up_proj': [k for k,v in model.named_modules() if k.endswith('mlp.up_proj')][a:b],
# }
# layer_groups

In [ ]:
# choose layers to cache
n_layers = model.config.num_hidden_layers
a = int(0.5*n_layers)
b = n_layers-2
select = slice(a, b, 3)
layer_groups = {
    'mlp.down_proj': [k for k,v in model.named_modules() if k.endswith('mlp.down_proj')][select],
    'self_attn': [k for k,v in model.named_modules() if k.endswith('.self_attn')][select],
    'mlp.up_proj': [k for k,v in model.named_modules() if k.endswith('mlp.up_proj')][select],
}
layer_groups

In [ ]:
import os, unicodedata, string
from pathlib import Path

def sanitize_path(path: Path | str, allow_period: bool = True) -> Path:
    """
    Whitelist only ASCII letters, digits, dash, underscore,
    optionally period, and forward‐slash. Replace others with '_'.
    """
    s = unicodedata.normalize("NFKD", str(path))\
                     .encode("ascii", "ignore")\
                     .decode()
    s = s.replace(os.sep, "/")
    allowed = set(string.ascii_letters + string.digits + "_-")
    if allow_period: allowed.add(".")
    allowed.add("/")
    return Path("".join(ch if ch in allowed else "_" for ch in s))

In [ ]:

acts_outfile = Path(f'/tmp/activation_store/ds_at-{model_name.replace("/", "")}-truthfulQA-bool-{split}-{len(ds2)}-{max_length}_v2.parquet')
acts_outfile = sanitize_path(acts_outfile)
acts_outfile

In [ ]:
def collect_all_tokens(*args, **kwargs):
    return default_postprocess_result(*args, **kwargs, last_token=False)


f = activation_store(ds, model, layers=layer_groups, postprocess_result=collect_all_tokens, 
                     outfile=acts_outfile
                     )
f

In [ ]:
# TODO which is better for mem, this or below?
ds_a = load_dataset("parquet", split='train', data_files=str(f), keep_in_memory=False).with_format("torch")
ds_a

In [ ]:
# # TODO which is better for mem, this or above?
# ds_a = Dataset.from_parquet(str(f), split=split, keep_in_memory=False).with_format("torch")
# ds_a

In [ ]:
act_groups = [c for c in ds_a.column_names if c.startswith('acts-')]
act_groups

In [ ]:
ds_a

In [ ]:
for k,v in ds_a[0].items():
    if hasattr(v, 'shape'):
        print(k, v.shape)
    else:
        print(k, type(v))

In [ ]:
# sanity test generate
b = next(iter(ds))
b = {k: v.to(model.device) for k, v in b.items()}
o = model.generate(
    inputs=b["input_ids"],
    attention_mask=b["attention_mask"],
    max_new_tokens=10,
)
gent = tokenizer.batch_decode(o, skip_special_tokens=False)
for g in gent:
    print(g)
    print("---")
    break

## Get supressed activations

In [ ]:
@torch.no_grad()
def get_supressed_activations(
    hs: Float[Tensor, "l b t h"], w_out, w_inv
) -> Float[Tensor, "l b t h"]:
    """
    Novel experiment: Here we define a transform to isolate supressed activations, where we hypothesis that style/concepts/scratchpads and other internal only representations must be stored.

    See the following references for more information:

    - https://arxiv.org/pdf/2401.12181
        - > Suppression neurons that are similar, except decrease the probability of a group of related tokens
        - > We find a striking pattern which is remarkably consistent across the different seeds: after about the halfway point in the model, prediction neurons become increasingly prevalent until the very end of the network where there is a sudden shift towards a much larger number of suppression neurons.

    - https://arxiv.org/html/2406.19384
        - > Previous work suggests that networks contain ensembles of “prediction" neurons, which act as probability promoters [66, 24, 32] and work in tandem with suppression neurons (Section 5.4).


    Output:
    - supression amount: This is a tensor of the same shape as the input hs, where the values are the amount of suppression that occured at that layer, and the sign indicates if it was supressed or promoted. How do we calulate this? We project the hs using the output_projection, look at the diff from the last layer, and then project it back using the inverse of the output projection. This gives us the amount of suppression that occured at that layer.
    """
    hs_flat = rearrange(hs[:, :, -1:], "l b t h -> (l b t) h")
    hs_out_flat = torch.nn.functional.linear(hs_flat, w_out)
    hs_out = rearrange(
        hs_out_flat, "(l b t) h -> l b t h", l=hs.shape[0], b=hs.shape[1], t=1
    )
    diffs = hs_out[:, :, :].diff(dim=0)
    diffs_flat = rearrange(diffs, "l b t h -> (l b t) h")
    # W_inv = get_cache_inv(w_out)

    # get the supression projected back
    supr_inv_flat = torch.nn.functional.linear(diffs_flat.to(dtype=w_inv.dtype), w_inv)
    supr_amounts = rearrange(
        supr_inv_flat, "(l b t) h -> l b t h", l=hs.shape[0] - 1, b=hs.shape[1], t=1
    ).to(w_out.dtype)

    # add on missing first layer
    torch.zeros_like(supr_amounts[:1]).to(hs.device)
    supr_amounts = torch.cat(
        [torch.zeros_like(supr_amounts[:1]).to(hs.device), supr_amounts], dim=0
    )
    return supr_amounts

In [ ]:
def get_uniq_token_ids(tokens):
    token_ids = tokenizer(
        tokens, add_special_tokens=False, padding=False
    ).input_ids
    token_ids = torch.tensor(list(set([x[0] for x in token_ids]))).long()
    print("before", tokens)
    print("after", tokenizer.batch_decode(token_ids))
    return token_ids


false_tokens = ["0", "0 ", "0\n", "false", "False "]
false_token_ids = get_uniq_token_ids(false_tokens)

true_tokens = ["1", "1 ", "1\n", "true", "True "]
true_token_ids = get_uniq_token_ids(true_tokens)

print('QC: manually check that these are equivilent (no <end_of_text> or newline)')

In [ ]:
# now we map to 1) calc supressed activations 2) llm answer (prob of 0 vs prob of 1)

Wo = model.get_output_embeddings().weight.detach().clone().cpu()
Wo_inv = torch.pinverse(Wo.clone().float())


def postprocess_activation_ds_rows(o):
    # TODO batch it
    """Process model outputs"""

    # get llm ans
    log_probs = o["logits"][-1].log_softmax(0)
    false_log_prob = log_probs.index_select(0, false_token_ids).sum()
    true_log_prob = log_probs.index_select(0, true_token_ids).sum()
    o["llm_ans"] = torch.stack([false_log_prob, true_log_prob])
    o["llm_log_prob_true"] = true_log_prob - false_log_prob

    # get supressed activations
    hs = o["hidden_states"][None]
    hs = rearrange(hs, "b l t h -> l b t h")
    supr_amounts = get_supressed_activations(hs, Wo.to(hs.dtype), Wo_inv.to(hs.dtype))

    # we will only take the last half of layers, and the last token
    layer_half = hs.shape[0] // 2
    
    hs = rearrange(hs, "l b t h -> b l t h").squeeze(0)[layer_half:-2]
    supr_amounts = rearrange(supr_amounts, "l b t h -> b l t h").squeeze(0)[layer_half:-2]

    o["hidden_states"] = hs.half()
    o["supr_amounts"] = supr_amounts.half()
    o['logits'] = o['logits'][-1].half()
    return o


ds_a2 = ds_a.map(postprocess_activation_ds_rows, writer_batch_size=1, num_proc=None)
ds_a2

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

In [ ]:
model = Wo = Wo_inv = tokenizer = None
clear_mem()

In [ ]:
{k: v.shape for k,v in ds_a2[0].items() if isinstance(v, torch.Tensor)}


In [ ]:
ds_a2

## Predict

In [ ]:
test_fraction = 0.2
TRAIN_TEST_SPLIT = int(max_length * (1- test_fraction))
TRAIN_TEST_SPLIT

# Or Skorch

In [ ]:
import numpy as np

from torch import nn

from skorch import NeuralNetRegressor
from skorch.toy import make_regressor

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:

def train_linear_prob_on_dataset(
    X,
    y,
    name="",
    device: str = "cuda",
    batch_size=32,
):
    # flatten
    X = X.flatten(1, -1).to(device)
    # X = X.view(len(X), -1).to(device)

    # norm X
    X = ((X - X.mean()) / X.std())
    if X.ndim == 1:
        X = X.unsqueeze(1)
    if y.ndim == 1:
        y = y.unsqueeze(1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # data.shape


    lr_model = NeuralNetRegressor(
        make_regressor(num_hidden=0, dropout=0, input_units=X.shape[-1]),
        lr=0.01,
        max_epochs=40,
        batch_size=batch_size,
        device='cuda',  # uncomment this to train with CUDA
        optimizer=torch.optim.Adam,
        optimizer__weight_decay=0.001,
        verbose=0,
    )
    # lr_model = Classifier(X.shape[-1], device=device)
    lr_model.fit(X_train, y_train)

    y_pred = lr_model.predict_proba(X_test)

    score = roc_auc_score(y_test.detach().cpu().numpy(), y_pred)
    logger.info(f"score for probe({name}): {score:.3f} roc auc, n={len(X_test)}. X.shape={X.shape}")
    return score

### transforms and filter

In [ ]:
# import jaxtyping
# %load_ext jaxtyping
# %jaxtyping.typechecker beartype.beartype  # or any other runtime type checker

In [ ]:
# https://github.com/pytorch/pytorch/issues/64947#issuecomment-2304371451
from math import ceil, floor

import torch


def torch_quantile(
    input: torch.Tensor,
    q: float | torch.Tensor,
    dim: int | None = None,
    keepdim: bool = False,
    *,
    interpolation: str = "nearest",
    out: torch.Tensor | None = None,
) -> torch.Tensor:
    """Better torch.quantile for one SCALAR quantile.

    Using torch.kthvalue. Better than torch.quantile because:
        - No 2**24 input size limit (pytorch/issues/67592),
        - Much faster, at least on big input sizes.

    Arguments:
        input (torch.Tensor): See torch.quantile.
        q (float): See torch.quantile. Supports only scalar input
            currently.
        dim (int | None): See torch.quantile.
        keepdim (bool): See torch.quantile. Supports only False
            currently.
        interpolation: {"nearest", "lower", "higher"}
            See torch.quantile.
        out (torch.Tensor | None): See torch.quantile. Supports only
            None currently.
    """
    # Sanitization: q
    try:
        q = float(q)
        assert 0 <= q <= 1
    except Exception:
        raise ValueError(f"Only scalar input 0<=q<=1 is currently supported (got {q})!")

    # Sanitization: dim
    # Because one cannot pass  `dim=None` to `squeeze()` or `kthvalue()`
    if dim_was_none := dim is None:
        dim = 0
        input = input.reshape((-1,) + (1,) * (input.ndim - 1))

    # Sanitization: inteporlation
    if interpolation == "nearest":
        inter = round
    elif interpolation == "lower":
        inter = floor
    elif interpolation == "higher":
        inter = ceil
    else:
        raise ValueError(
            "Supported interpolations currently are {'nearest', 'lower', 'higher'} "
            f"(got '{interpolation}')!"
        )

    # Sanitization: out
    if out is not None:
        raise ValueError(f"Only None value is currently supported for out (got {out})!")

    # Logic
    k = inter(q * (input.shape[dim] - 1)) + 1
    out = torch.kthvalue(input, k, dim, keepdim=True, out=out)[0]

    # Rectification: keepdim
    if keepdim:
        return out
    if dim_was_none:
        return out.squeeze()
    else:
        return out.squeeze(dim)

    return out

In [ ]:
from typing import Tuple
FiltIn = Float[Tensor, 'B L T H']
FiltOut = Tuple[FiltIn, FiltIn]

In [ ]:
def transform_hs_sup(o: Dataset, thresh: float = 1.0e-2) -> FiltOut:
    """
    Calc supressed activations for a certain threshold
    """
    supr_amounts = o["supr_amounts"]
    hs = o["hidden_states"] # [b l h]
    if thresh > 0:
        supressed_mask = (supr_amounts > thresh).to(hs.dtype)# [b l h]
    else:
        supressed_mask = (supr_amounts < thresh).to(hs.dtype)
    return hs * supressed_mask
    # return {
    #     f'supressed_hs_{thresh}':hs * supressed_mask
    # }

# for eps in [-10, -5, -1, -0.5, -0.1, -0.01, -0, 0, 0.01, 0.1, 0.5, 1, 10]:
#     ds_a2 = ds_a2.map(calc_hs_sup, fn_kwargs={'eps': eps}, writer_batch_size=1, num_proc=None)

In [ ]:


def magnitude_filtered_post_softmax(x: FiltIn, quantile=0.9) -> FiltOut:
    """Filter out tokens with abnormally high post-softmax values"""
    # Apply softmax to get attention-like weights
    weights = torch.softmax(x, dim=-1).max(dim=-1, keepdim=True).values
    
    # Create mask for tokens below threshold
    # FIXME should be independant of batch
    threshold = torch_quantile(weights, quantile)
    mask = (weights <= threshold)
    
    # # Ensure we don't filter everything out
    if mask.sum() == 0:
        logger.warning(f"All tokens filtered out threshold={threshold}.")
    #     # Keep all but the highest attention token
    #     _, max_idx = weights.max(dim=2)
    #     mask = torch.ones_like(weights, dtype=torch.bool)
    #     mask[max_idx] = False
    
    return x * mask, mask

In [ ]:
def quantile_filtered(x: FiltIn, quantile=.9) -> FiltOut:
    """Filter out tokens with attention weights above a quantile threshold"""
    weights = torch.log_softmax(x, dim=-1)#.max(dim=-1, keepdim=True).values
    # can't use max as there are too many attention sinks
    # print(weights)
    # FIXME should be independant of batch
    threshold = torch_quantile(weights, quantile)
    mask = weights <= threshold
    # print(weights.shape, mask.shape, mask.float().mean(), threshold)
    if mask.sum() == 0:
        logger.warning(f"All tokens filtered out quantile={quantile}.")
    return x * mask, mask

In [ ]:
# X = ds_a2['hidden_states']
# print(X.shape)
# X2, mask = quantile_filtered(X)
# mask.float().mean()

In [ ]:
import math

def entropy_guided_filter(x: FiltIn, quantile=0.9, normalize=True) -> FiltOut:
    """Filter tokens with entropy-based threshold using mean/std statistics
    
    Args:
        x: Input tensor of shape [tokens, hidden_dim]
        z_threshold: How many standard deviations from mean to use as threshold
        
    Returns:
        Filtered tensor with high-attention tokens removed
    """
    # Get attention-like weights
    p = torch.softmax(x, dim=-1)
    entropy_per_token = -(p * torch.log(p + 1e-8)).sum(dim=-1, keepdim=True)
    
    # optionally normalize by max entropy (log of feature dimension)
    if normalize:
        denom = math.log(p.shape[-1])
        entropy_per_token = entropy_per_token / denom
    
    # Get statistics across tokens
    # FIXME should be independant of batch
    threshold = torch_quantile(entropy_per_token, quantile)
    
    # Create mask for tokens below threshold
    mask = entropy_per_token <= threshold
    
    # Ensure we don't filter everything
    if mask.sum() == 0:
        logger.warning(f"All tokens filtered out z_threshold={quantile}.")
        # mask = torch.ones_like(weights, dtype=torch.bool)
        # mask[entropy_per_token.argmax()] = False
    
    return x * mask, mask

In [ ]:
# X = ds_a2['hidden_states']
# X2, mask = entropy_guided_filter(X)
# X.shape, X2.shape, X.norm(), X2.norm(), mask.shape, mask.float().mean()

In [ ]:
# Add new reduction functions that filter out potential attention sinks
def filter_special_positions(x: FiltIn, positions_to_exclude=[0, -1]) -> FiltIn:
    """Filter out specific positions like first (BOS) and last token"""
    mask = torch.ones(x.shape, dtype=torch.bool, device=x.device)
    for pos in positions_to_exclude:
        if pos < 0:
            actual_pos = x.shape[0] + pos
        else:
            actual_pos = pos
        if 0 <= actual_pos < x.shape[0]:
            mask[actual_pos] = False
    return x * mask, mask

def filter_high_magnitude(x: FiltIn, threshold_factor=2.0) -> FiltIn:
    """Filter out tokens with abnormally high magnitude (potential attention sinks)"""
    magnitudes = torch.norm(x, dim=-1, keepdim=True)

    # FIXME should be independant of batch
    mean_mag = magnitudes.mean()
    std_mag = magnitudes.std()
    threshold = mean_mag + threshold_factor * std_mag
    mask = magnitudes <= threshold
    if mask.sum() > 0:  # Ensure we don't filter everything
        return x * mask, mask
    else:
        # Fallback: keep all but the highest magnitude
        _, sorted_indices = torch.sort(magnitudes, descending=True)
        mask = torch.ones_like(magnitudes, dtype=torch.bool)
        mask[sorted_indices[0]] = False
        return x * mask, mask
    

In [ ]:
# X = ds_a2['hidden_states']
# X2 = filter_high_magnitude(X)
# X.shape, X2.shape, X.norm(), X2.norm()

## Define permutations of token_reductions, datasets, filters

In [ ]:
X_cols = ["supr_amounts", "hidden_states",] + act_groups

datasets = {}

# plain cols
for c in X_cols:
    datasets[c] = lambda ds_a2: ds_a2[c]

# differen't suppressed activations
for eps in [-5, -1, -0.5, -0.1, -0.01, -0, 0, 0.01, 0.1, 0.5, 1, 5]:
    datasets[f'supressed_hs_{eps}'] = lambda ds_a2: transform_hs_sup(ds_a2, eps)


# filters/transformers which we apply to all
filters = {
    "special": filter_special_positions,
    "magnitude": filter_high_magnitude,
    "entropy": entropy_guided_filter,
    "quantile": quantile_filtered,
}
# also diff magnitude filters
for eps in [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]:
    filters[f'magnitude_{eps}'] = lambda x: filter_high_magnitude(x, eps)

# # 2. token aggregators
# token_level_funcs = {
#     "min:": lambda x: x.min(0)[0],
#     "max": lambda x: x.max(0)[0],
#     "mean": lambda x: x.mean(0),
#     "sum": lambda x: x.sum(0),
#     "first": lambda x: x[0],
#     "last": lambda x: x[-1],
#     # "none": lambda x: x,
#     "std": lambda x: x.std(0),
# }
# unit test agg
token_level_funcs = {
    "min:": lambda x: x.min(2)[0],
    "max": lambda x: x.max(2)[0],
    "mean": lambda x: x.mean(2),
    "sum": lambda x: x.sum(2),
    "first": lambda x: x[:, :, 0],
    "last": lambda x: x[:, :, -1],
    # "none": lambda x: x,
    "flatten": lambda x: x.flatten(2),
    "std": lambda x: x.std(2),
}

# now get and shuffle all perms
perms = []
for k, _ in datasets.items():
    for k2, _ in filters.items():
        for k3, _ in token_level_funcs.items():
            perms.append((k, k2, k3))


perms = list(perms)
random.Random(42).shuffle(perms)
perms[:2]

In [ ]:
# # unit test filter
# X0 = ds_a2['hidden_states']
# for k,v in filters.items():
#     X0_norm = X0.norm()
#     X2, mask = v(X0) # bad
#     print(f"filter={k}\n\tmask.mean()={mask.float().mean():2.2%} \n\tmask={mask.shape}, \n\toutput={X0.shape}->{X2.shape}, \n\tnorm={X0_norm}->{X2.norm():2.6f}={(X0_norm-X2.norm())/X0_norm:.6%}")
#     assert mask.float().mean() < 1.0
#     assert mask.float().mean() > 0.0
#     assert X2.shape == X0.shape
#     assert X2.norm() < X0_norm



In [ ]:
output_path = Path('../outputs')
output_path.mkdir(exist_ok=True, parents=True)

In [ ]:
results = []
for i, (ds_key, filter_key, token_key) in tqdm(enumerate(perms), total=len(perms)):

    name = f"{ds_key}_{filter_key}_{token_key}"
    res_f = output_path / f"{acts_outfile.stem}_{ds_key}_{filter_key}_{token_key}.json"
    res_f = sanitize_path(res_f)
    if res_f.exists():
        d = json.load(res_f.open())
        logger.info(f'Already processed {res_f}, skipping {d["score"]:2.2f}')
        results.append((name, d["score"]))
        continue

    try:
        # TODO I would also like to cache the results
        logger.info(f"Processing {ds_key}, {filter_key}, {token_key}")
        ds = datasets[ds_key]
        filter_func = filters[filter_key]
        token_func = token_level_funcs[token_key]

        # get the data
        X = ds(ds_a2)
        logger.info(f"ds.shape: {X.shape}")
        X, mask = filter_func(X)
        logger.info(f"Xfilt.shape: {X.shape}")
        X = token_func(X)
        logger.info(f"Xtkn.shape: {X.shape}")

        # train the model
        y = ds_a2["label"].to('cuda').float()
        score = train_linear_prob_on_dataset(X, y, name=f"{ds_key}_{filter_key}_{token_key}")
        results.append((ds_key, filter_key, token_key, score))

        res = {
            "ds_key": ds_key,
            "filter_key": filter_key,
            "token_key": token_key,
            "score": score,
        }
        json.dump(res, open(res_f, "w"))
        results.append((name, score))
    except KeyboardInterrupt:
        logger.info(f"KeyboardInterrupt, stopping {ds_key}, {filter_key}, {token_key}")
        break
    except Exception as e:
        logger.error(f"Error processing {ds_key}, {filter_key}, {token_key}: {e}")
        raise
        continue

    clear_mem()

## Add baselines (llm_ans_ logits)

In [ ]:
# name = "logits"
# X = ds_a2['logits']
# score = train_linear_prob_on_dataset(X, name)
# results.append((name, score))
# score

In [ ]:

X = ds_a2['llm_ans'].exp()
# X = X[:, 1] / (X[:, 0] + X[:, 1])
X = X.argmax(1)
y = ds_a2['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_fraction, shuffle=False)

score = roc_auc_score(y_test, X_test).item()
if score<0.5:
    score = 1-score
results.append(('llm_ans', score))
score

In [ ]:
X = torch.sigmoid(ds_a2['llm_log_prob_true']/10) # would be better to calibrate or logreg
y = ds_a2['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_fraction, shuffle=False)

score = roc_auc_score(y_test, X_test).item()
results.append(('llm_log_prob_true', score))
score

## View results

In [ ]:
import pandas as pd

df = pd.DataFrame(results, columns=["name", "auroc"]).sort_values(
    "auroc", ascending=False
)
print(model_name, "Top results")
print(df.head(20).to_markdown())

In [ ]:
df['data'] = df['name'].apply(lambda x: x.split()[0])
# FIXME this is not keeping name and auroc paired
# df['reduction'] = df['name'].apply(lambda x: x.split()[-1])
df2 = df.groupby('data').apply(lambda g: g.sort_values("auroc", ascending=False).iloc[0], include_groups=False).sort_values("auroc", ascending=False)
print('top reduction for each data type')
print(df2.to_markdown())

In [ ]:
# plot it

from matplotlib import pyplot as plt
from pathlib import Path
import seaborn as sns
sns.set_theme()

c = ['llm_ans', 'llm_log_prob_true', 'hidden_states',  'supressed_hs'] + act_groups
df3 = df2.T[c].rename(columns={
    'llm_ans': 'LLM Answer',
    'llm_log_prob_true': 'LLM Probability',
    'hidden_states': 'Hidden States',
    'acts': 'Activations: up_proj',
    # 'logits': 'Logits',
    'supressed_hs': 'Supressed Hidden States',
}).T.sort_values("auroc", ascending=False)
# df3.plot.barh()
sns.barplot(data=df3, x='auroc', y=df3.index)
plt.legend().remove()
plt.xlabel("Linear probe AUROC")
plt.title(f"TruthfulQA Binary with {model_name}")
plt.xlim(0.5, None)
f = Path('../figs/').joinpath(f"truthfulqa_{model_name.replace('/', '_')}.png")
plt.savefig(str(f), bbox_inches='tight')
f